# Enable Logging for LLM Calls📜

Ensure security, audit, and compliance by enabling logging for all calls made to **LLMs**. Keep track of every interaction and maintain a transparent record for auditing purposes.🔒

### Import all needed packages

In [1]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [2]:
import boto3
import json
import os
import datetime
from botocore.exceptions import ClientError


class CloudWatch_Helper: 

    def __init__(self):
        # Create a Boto3 client for the CloudWatch Logs service     
        self.cloudwatch_logs_client = boto3.client('logs', region_name="us-west-2")

    def create_log_group(self, log_group_name):
        try:
            response = self.cloudwatch_logs_client.create_log_group(logGroupName=log_group_name)
            print(f"Log group '{log_group_name}' created successfully.")
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
                print(f"Log group '{log_group_name}' already exists.")
            else:
                print(f"Failed to create log group '{log_group_name}'. Error: {e}")

    def print_recent_logs(self, log_group_name, minutes=5):
        try:
            # Calculate the time range
            end_time = int(datetime.datetime.now().timestamp() * 1000)  # Current time in milliseconds
            start_time = end_time - (minutes * 60 * 1000)  # 5 minutes ago in milliseconds

            # Fetch log streams (assumes logs are stored in streams within the log group)
            streams = self.cloudwatch_logs_client.describe_log_streams(
                logGroupName=log_group_name,
                orderBy='LastEventTime',
                descending=True
            )

            for stream in streams.get('logStreams', []):
                # Fetch log events from each stream
                events = self.cloudwatch_logs_client.get_log_events(
                    logGroupName=log_group_name,
                    logStreamName=stream['logStreamName'],
                    startTime=start_time,
                    endTime=end_time
                )

                for event in events.get('events', []):
                    try:
                        # Try to load the string as JSON
                        json_data = json.loads(event['message'])
                        # Pretty print the JSON data
                        print(json.dumps(json_data, indent=4))
                    except json.JSONDecodeError:
                        # If it's not valid JSON, print the original string
                        print(event['message'])
                    print(f'{"-"*25}\n')

        except ClientError as e:
            print(f"Error fetching logs: {e}")


cloudwatch = CloudWatch_Helper()

In [3]:
log_group_name = '/my/amazon/bedrock/logs'

In [4]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' created successfully.


In [5]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [6]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': '12ef6935-d818-41e4-bedd-40771a4f9c95',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 15 Oct 2024 19:40:09 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '12ef6935-d818-41e4-bedd-40771a4f9c95'},
  'RetryAttempts': 0}}

In [7]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': '3d70c37a-b823-4261-bb61-e6cc7bd591fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 15 Oct 2024 19:40:11 GMT',
   'content-type': 'application/json',
   'content-length': '604',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3d70c37a-b823-4261-bb61-e6cc7bd591fa'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::242772082811:role/c99355a2566044l7970939t1w24277208281-LoggingIAMRole-O9stAXw6iXdt',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l7970939t1w242772082-loggings3bucket-navjlps0fgkc',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l7970939t1w242772082-loggings3bucket-navjlps0fgkc',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [8]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [9]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet that has been featured in various works of science fiction and fantasy literature. It is a planet that is inhabited by a wide range of creatures, including humans, aliens, and creatures that are not known to exist on Earth.

One of the most notable features of Foobar is its geography. The planet is shaped like a giant sphere, with a diameter of over 1,000 miles. The surface of the planet is covered in a variety of landscapes, including deserts, forests, and oceans. The oceans of Foobar are known to be deep and dangerous, with many dangerous creatures living in them.

The creatures of Foobar are also diverse and unique. Some of the most common creatures include humans, who are the dominant species on the planet. Humans on Foobar have a variety of cultures and beliefs, and they have developed advanced technology and science. Other creatures include aliens, who are beings that come from other planets or galaxies. Aliens on Foobar come in a wide range of shape

In [10]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-10-15T19:40:13Z",
    "accountId": "242772082811",
    "identity": {
        "arn": "arn:aws:sts::242772082811:assumed-role/voclabs/user3590050=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJhcHA"
    },
    "region": "us-west-2",
    "requestId": "6953bf85-3a6b-4491-a4ca-e771a74f67c7",
    "operation": "InvokeModel",
    "modelId": "amazon.titan-text-express-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "Write an article about the fictional planet Foobar.",
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        },
        "inputTokenCount": 10
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJ

To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [11]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [12]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
